# IMPORTS

In [1]:
import json
import os
import time
from AxonDeepSeg.config import *

## Setting hyperparameters and creating the file _config_

Quick explanations

In [2]:
# Description du fichier config :
# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].

# network_features_per_convolution : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[[64,64] for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_convolution[k][1] = network_features_per_convolution[k+1][0].

# network_trainingset : string : String describing the dataset used for the training.

Name of the .json file where the network configuration will be stored

In [3]:
filename = '/config_network.json'

## Setting the main parameters of the network

If multiple training sets are available, you can select which one you want to use here.

In [4]:
trainingset_name = 'SEM_2classes_reduced'

We then set the main parameters and the architecture of the network

In [5]:
network_learning_rate = 0.001
network_n_classes = 2
dropout = 0.75
network_depth = 4
network_convolution_per_layer = [3 for i in range(network_depth)]
network_size_of_convolutions_per_layer = [[5,5,5],[3,3,3],[3,3,3],[3,3,3]]#[[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]
network_features_per_convolution = [[[1,10],[10,20],[20,30]],[[30,40],[40,50],[50,60]],[[60,70],[70,80],[80,90]],[[90,100],[100,110],[110,120]],
                                    ]
network_batch_size = 8
network_batch_norm_decay = 0.90
network_batch_norm = True
downsampling = 'convolution'


# Note: type can be all (we will do all transformations set to True (default) on each patch) or random (we will choose one random transformation
# (among the ones set to True) and apply each to the current patch)
# You can also set 'type' to 'none' (string), then we won't do any transformation.

network_data_augmentation = {'type':'all', 
                                      'transformations':{'shifting':False, 'rescaling':False,
                                                         'random_rotation':True, 'elastic':False, 'flipping':False,
                                                         'noise_addition':False}
                            }



thresholds = [0,0.5]

weighted_cost = False

config = {
    'network_n_classes': network_n_classes,
    'network_thresholds': thresholds,
    'network_learning_rate': network_learning_rate,
    'network_batch_size': network_batch_size,
    'network_dropout': dropout,
    'network_depth': network_depth,
    'network_batch_norm_decay': network_batch_norm_decay,
    'network_convolution_per_layer': network_convolution_per_layer,
    'network_size_of_convolutions_per_layer': network_size_of_convolutions_per_layer,
    'network_features_per_convolution': network_features_per_convolution,
    'network_trainingset': trainingset_name,
    'network_downsampling': downsampling,
    'network_weighted_cost': weighted_cost,
    'network_batch_norm': network_batch_norm,
    'network_data_augmentation': network_data_augmentation
}




## Preparing the training phase

In [6]:
path_training = '../data/'+trainingset_name+'/training/'

## Actually training the model

By default, the name is the current date. You can uncomment the second line dir_name if you want to give a specific name to the folder, but beware: it will erase the contents of the folder if it already exists !

The model data will be stored in a folder with this name, as well as the summaries that are going to be readable on tensorboard.

In [7]:
dir_name = time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S") 
dir_name = 'cv_2c_nw_d5_2_b8_p10-10'
path_model = os.path.join('../models/', dir_name)

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
path_model_init = None 


# Specify here the path to the initial model if needed.

if os.path.exists(path_model+filename): # if there is already a configfile
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_model+filename, 'w') as f:
        json.dump(config, f, indent=2)
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
        
        
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config_network)
    
################################################### Training phase ###################################################

# Note : possible values for gpu: None, '/gpu:0'. If used withing CUDA_VISIBLE_DEVICES=?, then it will use gpu number ? by default
from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network,path_model_init=path_model_init,gpu='/gpu:0')

None
('Input features layer : ', 220)
('Input features layer : ', 180)
('Input features layer : ', 140)
('Input features layer : ', 100)
('Input features layer : ', 60)
('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 10])
('Size:', 3)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [20, 30])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 0, 'Features: ', [30, 40])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 1, 'Features: ', [40, 50])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 0, 'Features: ', [50, 60])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 1, 'Features: ', [60, 70])
('Size:', 3)
('Layer: ', 3, ' Conv: ', 0, 'Features: ', [70, 80])
('Size:', 3)
('Layer: ', 3, ' Conv: ', 1, 'Features: ', [80, 90])
('Size:', 3)
('Layer: ', 4, ' Conv: ', 0, 'Features: ', [90, 100])
('Size:', 3)
('Layer: ', 4, ' Conv: ', 1, 'Features: ', [100, 110])
('Size:', 3)
('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 10])
('Size:', 3)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [20, 30])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 0, 'Features

/home/GRAMES.POLYMTL.CA/mawab/ads/tf11-ads/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyboardInterrupt: 